# 2.07 syncronize voices



In [4]:
from pyknon.genmidi import Midi
from pyknon.music import Rest, Note, NoteSeq
import numpy as np
import matplotlib.pyplot as plt

__Instruments:__
Available are at lest the 128 General-Midi (GM) Instruments. Depending on the sound-fonts there is a bigger choise. A list of the GM instruments can be found here. https://jazz-soft.net/demo/GeneralMidi.html


In [5]:
major = np.array([ 0, 2, 4, 5, 7, 9, 11])
minor = np.array([ 0, 2, 3, 5, 7, 8, 10])  
C7 = np.array([ 0, 4, 7, 10]) 
CM7 = np.array([ 0, 4, 7, 11])
Cm7 = np.array([ 0, 3, 7, 10])
Cm  = np.array([ 0, 3, 7])
C   = np.array([ 0, 4, 7])
bass= np.array([ 0])

In [111]:

def scale_create(tones):
    tones = np.asarray(tones)   # tones which form chord or scale in the first octave (0-11)
    if any(tones > 11):             # tones over one octave?
        tones = np.mod(tones,12)    # set the thones in one octave
        tones = np.sort(tones)      # sort the tones new
        tones = np.unique(tones)    # remove duplicate tones
    octave = np.repeat( np.linspace(0,108, num=10), len(tones))
    scale = np.add( octave, np.tile(tones, 10)) # add element wise octave and note
    return scale.astype(int)
    
def fade(start,end,steps): # currently not in use
    fade = np.around( np.linspace(start,end,num=steps))
    fade = fade.astype(int)
    return fade

def ran_duration(duration, prob_duration, melody_len):    
    duration= np.asarray(duration)                  # this are the allowed durations of the notes
    prob_duration = np.asarray(prob_duration)       # this are the probabilities how often each will occure
    prob_duration = prob_duration/np.sum(prob_duration)
    cumsum, melody_len, rythem = 0, melody_len/4 , np.asarray([])  #melody_len/4 as note values are quarter
    while cumsum < melody_len:
        note_len = np.random.choice(duration, p=prob_duration)
        cumsum = cumsum + note_len
        rythem = np.append(rythem,note_len)
    return rythem , len(rythem)

def ran_volume(volume, prob_volume, melody_len):
    volume = np.asarray(volume, dtype=int)         # this are the allowed volumes of thenotes
    prob_volume = np.asarray(prob_volume)          # this are the probabilities how often each volume will occure
    prob_volume = prob_volume/np.sum(prob_volume) 
    volumes = np.r_[np.random.choice(volume, size=melody_len, p=prob_volume)]
    return volumes

#   currently not in use
def intvl_melody(intvl, prob_intvl, melody_len):  #Interval Melody   
    intvl = np.asarray(intvl)            # Possible interval
    prob_intvl = np.asarray(prob_intvl)         # Probability of each interval
    prob_intvl = prob_intvl/np.sum(prob_intvl)
    intervals = np.r_[np.random.choice(intvl, size=melody_len, p=prob_intvl)] 
    imelody = np.cumsum(intervals)
    return imelody


#   liniar_range: Generates an range in which the instrument can play. 
def liniar_range(r_start, r_top, r_edge, r_end, title): # acceptance range of the instrument 
    h = 100 # hight of acceptance function
    a_range = np.zeros(121, dtype=int)  # only to midi =120 as 127 is not a complete octave
    np.put(a_range, range(r_start,r_top),  np.linspace(0,h, num=(r_top -r_start)) )
    np.put(a_range, range(r_top, r_edge),  np.linspace(h,h, num=(r_edge-r_top  )) )
    np.put(a_range, range(r_edge, r_end),  np.linspace(h,0, num=(r_end -r_edge )) )
    
    fig, ax = plt.subplots()
    ax.plot(range(121), a_range)
    plt.title(str(title)+':  '+str([r_start, r_top, r_edge, r_end]))
    plt.show()
    return a_range
    
#   i_last_note: finds de i value of the last not in the actual scale.
def i_last_note(note, scale):
    i_note = (np.abs(scale - note)).argmin()
    return i_note

#   intvl_next is a modification of intvl_melody. But it does only creats one interval and not an array/melody in one time.
def intvl_next(intvl, prob_intvl):  #singel interval
    intvl = np.asarray(intvl)            # Possible interval
    prob_intvl = np.asarray(prob_intvl)         # Probability of each interval
    prob_intvl = prob_intvl/np.sum(prob_intvl)
    interval = np.random.choice(intvl, size=1, p=prob_intvl)
    return interval[0]

In [21]:
#   acceptance: accepts and refuses proposed nots with Metropolis-Hasting Algorythem.
#     x is the value in the aceptance range of the current note, while x_new is it from the proposoal note
def acceptance(x, x_new):
    if x_new < 1:
        if x < 1:
            print('start_note not in range')
            return 
    quot = x_new/x
    if quot >= 1: return True
    if np.random.uniform(0,1)< quot: return True
    else: return False
  

In [110]:
def pattern_gen(scales,melody_len):
    scales = np.asarray(scales)
    bpb = 4  # beats per bar
    factor = int(np.trunc(melody_len/(np.sum(scales[:,0]) * bpb)) + 1) # factor rounded up: how many times is the pattern used
    change_times = np.cumsum(np.tile(scales[:,0],factor)) * bpb        # create change time list with factor
    change_times = np.concatenate((np.asarray([0]),change_times))[:-1] # add 0 at beginig remove last element
    
    for i in range(len(scales)):         # send scales to scale_create
        scales[i,1] = scale_create(scales[i,1])
    pattern = np.tile(scales,(factor,1))   # tile the scales as long the melody is
    pattern[:,0] = change_times    #insert change_times into scales
    pattern = np.delete(pattern, np.argwhere(pattern[:,0] >= melody_len) ,0) # remove unneeded scales
    
    return pattern

In [117]:
def acceptance_melody(intvl, prob_intvl, pattern, start_note, a_range, notenr, rythem):
    melody = np.zeros(notenr, dtype=int)
    cum_rythem = np.cumsum(rythem) *4
    scale_change = pattern[:,0]
   ### melody[0] = scales[0][i_last_note(start_note,scales[0])]
    
    scale_nr =0   
    for npn in range(1, notenr+1):  #npn: note per note (index)
        
        print(np.argwhere(scale_change < 20))
        
        # -- scale changing ------
        if npn == pattern[:   # number in mp.mod: how many tones in one scale
            scale_nr = scale_nr + 1
            if scale_nr >= len(scales):
                scale_nr =0       
        scale = scales[scale_nr]
        
        
        accept = False    
        while not accept:       # aslong acept == False
            inote = i_last_note(melody[npn-1],scale)
            inote_next = inote + intvl_next(intvl, prob_intvl)         # add current not with Proposition
            accept_val = a_range[[melody[(npn-1)],scale[inote_next]]]  # get acceptance values
            accept = acceptance(accept_val[0],accept_val[1])
        melody[npn] = scale[inote_next]
    return melody
            
        

SyntaxError: invalid syntax (<ipython-input-117-e6e00b3c1820>, line 14)

__tune_S:__ 

In [103]:
def tune_S():
    tune_name = 'tune_S'  
    #np.random.seed(23)
    bar, bpb = 16, 4  # bar: Takt , bpb: beat per bar
    melody_len = bar * bpb

    #scales = [scale_create(CM7),scale_create(Cm7+9),scale_create(Cm7+2),scale_create(C7+7)] #rythem Change
    scales = [[4,C7],[2,C7+5],[2,C7],[1,C7+7],[1,C7+5],[2,C7]] # Blues 
    pattern = pattern_gen(scales, melody_len)
    print(pattern)
    
    
    rythem1, notenr_1 = ran_duration([1/8, 1/4,1/2], [4,2,1], melody_len)
    range_1 = liniar_range(30,48,72,84, tune_name +' Range 1')
    melody1 = acceptance_melody([-2,-1, 0, 1, 2],[1, 3, 1, 3, 1],pattern, 60, range_1, notenr_1, rythem1)
    volumes1 = ran_volume([0,120], [2,8], notenr_1 )
    notes1 = NoteSeq( [Note(no,octave=0, dur=du, volume=vo) for no,du,vo in zip(melody1,rythem1,volumes1)] )


    
    instruments = [0]
    notes = [notes1]
    return notes, instruments,tune_name

In [104]:

def gen_midi():
#     squezze into a MIDI framework
    notes, instruments, tune_name = tune_S() #  <--- select a tune  <<--     <<<<<<<<<--- select a tune -----
    nTracks = len(notes)
    
    m = Midi(number_tracks=nTracks, tempo=120, instrument=instruments)
    for iTrack in range(nTracks):
        m.seq_notes(notes[iTrack], track=iTrack)

    #--- write the MIDI file -----
    midi_file_name = tune_name +'.mid'   # set the name of the file
    m.write(midi_file_name)
    return midi_file_name

## Midi: Play and Generate audio-file 
Externel players offered a better sound quality in comparison with python liaberys. We use __VLC__  and __Musescore__. The __soundfont__ for the VLC player is defined over the command line. For Musescore through the Gui in the preferences.


In [105]:
import subprocess
default_soundfont = '/usr/share/sounds/sf3/MuseScore_General.sf3'

def midi_play(midi_in, soundfont= default_soundfont):
    subprocess.call(['cvlc', midi_in , 'vlc://quit', '--soundfont', '/home/viturin/-vitis/Documents/MuseScore2/Soundfonts/Compifont_13082016.sf2'])   # cvlc = vlc without gui
    
def midi_audio(midi_in, name_out = 'none', soundfont= default_soundfont):
    if name_out == 'none' :
        name_out = midi_in.replace('.mid', '.flac')
    else:
        name_out = name_out + '.flac'
    subprocess.call(['mscore', '-o', name_out, midi_in]) # -o = export as

def midi_png(midi_in, name_out = 'none'):
    if name_out == 'none' :
        name_out = midi_in.replace('.mid', '.png')
    else:
        name_out = name_out + '.png'
    subprocess.call(['mscore', '-o', name_out, '-T', '2', midi_in]) # -o = export as , -T 2 = cut page with 2 pixel

In [107]:
######---  Main  ---######
midi_file_name = gen_midi()

midi_play(midi_file_name)
midi_audio(midi_file_name)
midi_png(midi_file_name)

[[0
  array([  0,   4,   7,  10,  12,  16,  19,  22,  24,  28,  31,  34,  36,
        40,  43,  46,  48,  52,  55,  58,  60,  64,  67,  70,  72,  76,
        79,  82,  84,  88,  91,  94,  96, 100, 103, 106, 108, 112, 115,
       118])]
 [16
  array([  0,   3,   5,   9,  12,  15,  17,  21,  24,  27,  29,  33,  36,
        39,  41,  45,  48,  51,  53,  57,  60,  63,  65,  69,  72,  75,
        77,  81,  84,  87,  89,  93,  96,  99, 101, 105, 108, 111, 113,
       117])]
 [24
  array([  0,   4,   7,  10,  12,  16,  19,  22,  24,  28,  31,  34,  36,
        40,  43,  46,  48,  52,  55,  58,  60,  64,  67,  70,  72,  76,
        79,  82,  84,  88,  91,  94,  96, 100, 103, 106, 108, 112, 115,
       118])]
 [32
  array([  2,   5,   7,  11,  14,  17,  19,  23,  26,  29,  31,  35,  38,
        41,  43,  47,  50,  53,  55,  59,  62,  65,  67,  71,  74,  77,
        79,  83,  86,  89,  91,  95,  98, 101, 103, 107, 110, 113, 115,
       119])]
 [36
  array([  0,   3,   5,   9,  12,  15,  17,  21,

ValueError: too many values to unpack (expected 2)

In [116]:
print(len(np.arange(1,4)))

3
